本タスクは将来ユーザが購入しそうな商品を過去データを元に推薦するアルゴリズムの作成，である

In [245]:
import pandas as pd # データ分析ライブラリ
import numpy as np # 三角関数，対数関数，行列などの数学的な計算を行うライブラリ
import seaborn as sns # グラフ可視化ライブラリ
import matplotlib.pyplot as plt # グラフ可視化ライブラリ
from pathlib import Path # ファイルパス操作ライブラリ

In [246]:
# データセットのパス
DATASET_PATH = Path('/Users/akiyoshi-n/Documents/data_analysis/data_analysis/data')

In [247]:
df = pd.read_excel(f'{DATASET_PATH}/ec_site_purchasing_rate_prediction.xlsx')
print(df.head(5))

     user_id  product_id  event_type  ad           time_stamp
0  0000306_B  00004491_b           1  -1  2017/04/03 00:19:02
1  0000306_B  00004491_b           1  -1  2017/04/03 05:16:53
2  0000306_B  00004491_b           1  -1  2017/04/03 00:19:14
3  0000306_B  00004491_b           1  -1  2017/04/03 00:18:05
4  0000306_B  00004491_b           1  -1  2017/04/03 00:19:15


特徴量<br>
event_type: 商品に対してとった行動．0: 商品をカートに入れる，1: 詳細ページ閲覧，2: 商品の広告をクリック，3: 商品を購入<br>
ad: 購入した場合，それが広告経由かどうか．0: 広告経由でない，1: 広告経由，-1: そもそも購入してない<br>
time_stamp: イベントが起こった年月日時

In [248]:
print(df.shape)

(135451, 5)


データの取得期間を確認

In [249]:
# time_stampをdatatime型に変換
df['time_stamp'] = pd.to_datetime(df['time_stamp'])
# データの取得期間を確認
Max = max(df['time_stamp'])
Min = min(df['time_stamp'])
print('データの取得期間は' + str(Min) + 'から' + str(Max) + 'までです。')
print('期間は' + str(Max - Min) + 'です。')
print(df['time_stamp'].dtypes)

データの取得期間は2017-03-31 15:00:10から2017-04-30 14:59:41までです。
期間は29 days 23:59:31です。
datetime64[ns]


ユニークユーザー数を確認

In [250]:
print(len(df['user_id'].unique()))

3000


ユニーク商品数を確認

In [251]:
print(len(df['product_id'].unique()))

12131


とった行動の種類とその件数を確認

In [252]:
df['event_type'].value_counts()

event_type
1    125357
0      6880
3      2803
2       411
Name: count, dtype: int64

購入パターンの件数を確認

In [253]:
df['ad'].value_counts()

ad
-1    132648
 1      1721
 0      1082
Name: count, dtype: int64

In [254]:
# ユーザごとの行動を時系列順に並べる
# groupby('カラム名')を用いることで指定したカラムで同じ値を持つ者同士のグループが作成される
# apply(lambda x: x.sort_values('カラム名'))を用いることで指定したカラムの値に従ってソートする
# groupbyメソッドにより指定したカラムにおいてグルーム分けし，applyメソッドにより指定したカラムの値に従ってソートし，最終的に処理済みのデータを結合する
df_u = df.groupby('user_id').apply(lambda x: x.sort_values('time_stamp'))
df_u.head(5)

user_id  product_id  event_type  ad          time_stamp
user_id                                                                   
0000010_B 38468  0000010_B  00007375_b           1  -1 2017-04-02 12:18:15
          38464  0000010_B  00016319_b           1  -1 2017-04-02 12:22:12
          38463  0000010_B  00016319_b           1  -1 2017-04-02 12:22:25
          38465  0000010_B  00011168_b           1  -1 2017-04-02 12:25:53
          38466  0000010_B  00011599_b           1  -1 2017-04-02 12:30:17

In [255]:
# 今回はindexが設定されているためlocでユーザ名を指定することでデータを取得することができる
df_u.loc['0000010_B']

,user_id,product_id,event_type,ad,time_stamp
38468,0000010_B,00007375_b,1,-1,2017-04-02 12:18:15
38464,0000010_B,00016319_b,1,-1,2017-04-02 12:22:12
38463,0000010_B,00016319_b,1,-1,2017-04-02 12:22:25
38465,0000010_B,00011168_b,1,-1,2017-04-02 12:25:53
38466,0000010_B,00011599_b,1,-1,2017-04-02 12:30:17
38467,0000010_B,00011599_b,1,-1,2017-04-02 12:31:04
38486,0000010_B,00003683_b,1,-1,2017-04-19 10:58:10
38485,0000010_B,00003683_b,1,-1,2017-04-19 11:00:30
38469,0000010_B,00010114_b,1,-1,2017-04-19 11:05:55
38489,0000010_B,00013685_b,1,-1,2017-04-19 11:11:55


In [256]:
# ユーザが関わった商品数を確認
# ユーザID: '0000010_B'について調べる
products = df_u.loc['0000010_B']['product_id'].unique()
print(len(products))

13


In [257]:
# ユーザID: '0000010_B'がとった行動を確認
# 商品閲覧しかしていないことがわかる
print(df_u.loc['0000010_B']['event_type'].value_counts())

event_type
1    29
Name: count, dtype: int64


In [258]:
# ユーザが行動をとった日数を確認
# 集中的に行動を起こしていたのかがわかる
ymd = df_u.loc['0000010_B']['time_stamp'].apply(lambda x: x.date())
print(len(ymd.unique()))
# 2日間にわたって行動を起こしていたことがわかる

2


商品毎のとられた行動を時系列順に並べる

In [259]:
df_p = df.groupby('product_id').apply(lambda x: x.sort_values('time_stamp'))
df_p.head(5)

user_id  product_id  event_type  ad          time_stamp
product_id                                                                  
00000000_b 128206  0020976_B  00000000_b           1  -1 2017-04-26 02:06:53
           128207  0020976_B  00000000_b           1  -1 2017-04-26 02:07:18
00000001_b 124937  0009792_B  00000001_b           1  -1 2017-04-04 09:52:08
           124938  0009792_B  00000001_b           1  -1 2017-04-04 09:54:34
           124939  0009792_B  00000001_b           1  -1 2017-04-04 09:55:42

In [260]:
df_p.loc['00000000_b']

,user_id,product_id,event_type,ad,time_stamp
128206,0020976_B,00000000_b,1,-1,2017-04-26 02:06:53
128207,0020976_B,00000000_b,1,-1,2017-04-26 02:07:18


In [261]:
# 商品に対して行動を行ったユーザを確認
# 商品ID: '00000000_b'について調べる
users = df_p.loc['00000001_b']['user_id'].unique()
print(len(users))

5


In [262]:
# 商品に対して行われた行動の種類とその数
df_p.loc['00000001_b']['event_type'].value_counts()

event_type
1    28
0     1
3     1
Name: count, dtype: int64

In [263]:
# 曜日別に行われた行動の数を確認
df_p.loc['00000001_b']['time_stamp'].apply(lambda x: x.day_name()).value_counts()

time_stamp
Wednesday    13
Tuesday      11
Monday        3
Friday        2
Thursday      1
Name: count, dtype: int64

ユーザ×商品毎の行動を時系列順に並べる

In [264]:
df_up = df.groupby(['user_id', 'product_id']).apply(lambda x: x.sort_values('time_stamp'))
df_up.head(5)

user_id  product_id  event_type  ad  \
user_id   product_id                                                
0000010_B 00003227_b 38484  0000010_B  00003227_b           1  -1   
          00003683_b 38486  0000010_B  00003683_b           1  -1   
                     38485  0000010_B  00003683_b           1  -1   
          00007375_b 38468  0000010_B  00007375_b           1  -1   
          00008693_b 38482  0000010_B  00008693_b           1  -1   

                                    time_stamp  
user_id   product_id                            
0000010_B 00003227_b 38484 2017-04-19 11:45:17  
          00003683_b 38486 2017-04-19 10:58:10  
                     38485 2017-04-19 11:00:30  
          00007375_b 38468 2017-04-02 12:18:15  
          00008693_b 38482 2017-04-19 12:11:49

In [265]:
# ユーザーID'0000010_B'と商品ID'00010114_b'のデータを表示
df_up.loc[('0000010_B', '00010114_b')]

,user_id,product_id,event_type,ad,time_stamp
38469,0000010_B,00010114_b,1,-1,2017-04-19 11:05:55
38473,0000010_B,00010114_b,1,-1,2017-04-19 12:17:20
38471,0000010_B,00010114_b,1,-1,2017-04-19 12:17:35
38472,0000010_B,00010114_b,1,-1,2017-04-19 12:19:06
38470,0000010_B,00010114_b,1,-1,2017-04-19 12:19:40
38474,0000010_B,00010114_b,1,-1,2017-04-19 12:19:51


In [266]:
df_up.loc[('0000010_B', '00010114_b')]['event_type'].value_counts()

event_type
1    6
Name: count, dtype: int64

In [267]:
# ユーザーID'0000015_B'と商品ID'00000744_b'のデータで購入した中で広告経由かどうかを表示
df_up_0 = df_up.loc[('0000015_B', '00000744_b')]
condition = df_up_0['event_type'] == 3
print(df_up_0[condition]['ad'].value_counts())

ad
1    1
0    1
Name: count, dtype: int64


In [268]:
# ユーザーID'0000015_B'と商品ID'00000744_bに対して関わった年月
ymd = df_up_0['time_stamp'].apply(lambda x: x.date()).unique()
len(ymd)

2

学習用データと検証用データ作成

学習データは最初の半分でそのさらに前半半分を説明変数期間(1/4期間)，後半を目的変数期間とする．テストデータも同じである．

In [269]:
# データの期間が1ヶ月なので，最初の2ヶ月分を学習，後半２ヶ月をテストとする
end = max(df['time_stamp'])
start = min(df['time_stamp'])
interval = end - start
print('データの取得期間は' + str(start) + 'から' + str(end) + 'までです。')
print('期間は' + str(interval) + 'です。')

データの取得期間は2017-03-31 15:00:10から2017-04-30 14:59:41までです。
期間は29 days 23:59:31です。


In [270]:
print('訓練データの説明変数期間:',start,'~',start + interval/4,'までです。')
print('訓練データの目的変数期間:',start + interval/4,'~',start + interval/2,'までです。')
print('テストデータの説明変数期間:',start + interval/2,'~',start + interval*3/4,'までです。')
print('テストデータの目的変数期間:',start + interval*3/4,'~',end,'までです。')

訓練データの説明変数期間: 2017-03-31 15:00:10 ~ 2017-04-08 03:00:02.750000 までです。
訓練データの目的変数期間: 2017-04-08 03:00:02.750000 ~ 2017-04-15 14:59:55.500000 までです。
テストデータの説明変数期間: 2017-04-15 14:59:55.500000 ~ 2017-04-23 02:59:48.250000 までです。
テストデータの目的変数期間: 2017-04-23 02:59:48.250000 ~ 2017-04-30 14:59:41 までです。


In [271]:
# 訓練データとテストデータを作成
df_train = df[df['time_stamp'] < start + interval / 2]
df_train_X = df_train[df_train['time_stamp']<start + interval/4]
df_train_y = df_train[df_train['time_stamp']>=start + interval/4]
df_test = df[df['time_stamp'] >= start + interval / 2]
df_test_X = df_test[df_test['time_stamp']<start + interval*3/4]
df_test_y = df_test[df_test['time_stamp']>=start + interval*3/4]

In [272]:
# テストの目的変数の期間のみのデータになっている
df_test_y.head(5)

,user_id,product_id,event_type,ad,time_stamp
16,0000306_B,00004491_b,1,-1,2017-04-28 06:35:28
58,0000306_B,00010337_b,1,-1,2017-04-26 01:31:54
59,0000306_B,00010337_b,1,-1,2017-04-28 06:40:59
60,0000306_B,00010337_b,1,-1,2017-04-28 06:41:03
86,0000306_B,00004995_b,1,-1,2017-04-28 06:33:08


今回のタスクは将来ユーザが購入しそうな商品を推薦するアルゴリズムの作成である．<br>
具体的に言えば過去の行動履歴データを元にそれぞれのユーザに対して将来購入するような商品を予測するというものである．

### ユーザに関する特徴量の作成<br>
ユーザの活性度のような特徴量作成
> 各ユーザが関わったユニーク商品数<br>
> 各ユーザがなんらかの行動をとった日数<br>
> 各ユーザが商品の詳細ページを閲覧した回数<br>
これらの特徴量によって行動パターンの違いを表現することが狙い．

In [273]:
# 各ユーザが関わったユニークな商品数
# u_p: user_productの略
u_p = df_train_X.groupby('user_id').apply(lambda x: len(x['product_id'].unique())) # lambda　x のxは関数の引数で，apply適応の場合はdfが入る
# testデータにおけるユーザごとの商品数
u_p_test = df_test_X.groupby('user_id').apply(lambda x: len(x['product_id'].unique()))
u_p.head(5)

user_id
0000010_B     4
0000018_B     8
0000033_B    19
0000036_B     1
0000040_B    13
dtype: int64

In [274]:
# 各ユーザがなんらかの行動をとった日数
# grooupbyの後のapplyの中のxはグループ分けされた行列が入る．つまり，行列全体に対する処理が行う．普通のdfやSeriesの後のapplyの中のxは行毎に行う関数の処理が入る
u_d = df_train_X.groupby('user_id').apply(lambda x: len(x['time_stamp'].apply(lambda x: x.date()).unique())) # len関数の中のx['time_stamp']に対してapplyを適応
# testデータにおけるユーザごとの日数
u_d_test = df_test_X.groupby('user_id').apply(lambda x: len(x['time_stamp'].apply(lambda x: x.date()).unique()))
u_d.head(5)

user_id
0000010_B    1
0000018_B    3
0000033_B    4
0000036_B    1
0000040_B    4
dtype: int64

In [275]:
# 各ユーザが商品の詳細ページを閲覧した回数
# groupbyだけを適応してメソッドをその後適応させない場合はgroupby型データになる．メソッドを適応させるとdf型データになる
u_pv = df_train_X.groupby('user_id').apply(lambda x: (x['event_type']==1).sum())
# testデータにおけるユーザごとの閲覧回数
u_pv_test = df_test_X.groupby('user_id').apply(lambda x: (x['event_type']==1).sum())
u_pv.head(5)

user_id
0000010_B     6
0000018_B    10
0000033_B    87
0000036_B     1
0000040_B    43
dtype: int64

In [276]:
# 3つのデータを連結
u = pd.concat([u_p, u_d, u_pv], axis=1)
# testデータにおけるユーザごとの商品数，日数，閲覧回数
u_test = pd.concat([u_p_test, u_d_test, u_pv_test], axis=1)
u.head(5)

,0,1,2
user_id,,,
0000010_B,4,1,6
0000018_B,8,3,10
0000033_B,19,4,87
0000036_B,1,1,1
0000040_B,13,4,43


In [277]:
# カラム名を変更
u.columns = ['u_p', 'u_d', 'u_pv']
u_test.columns = ['u_p', 'u_d', 'u_pv']
u.head(5)

,u_p,u_d,u_pv
user_id,,,
0000010_B,4,1,6
0000018_B,8,3,10
0000033_B,19,4,87
0000036_B,1,1,1
0000040_B,13,4,43


In [278]:
u_test.head(5)

,u_p,u_d,u_pv
user_id,,,
0000010_B,9,1,23
0000015_B,1,1,1
0000018_B,4,4,7
0000041_B,2,1,1
0000045_B,11,2,23


### 商品に関する特徴量の作成
商品の人気度のような特徴量作成
> 各商品に対して何らかの行動をとったユニークユーザー数<br>
> 各商品に対して閲覧された回数<br>
> 各商品に対してカートに入れられた回数<br>
> 各商品に対してクリックされた回数<br>
> 各商品に対して購入された回数<br>
これらの特徴量によって各商品の人気度を表現することが狙い．

In [279]:
# 各商品に対して何らかの行動をとったユニークなユーザ数
p_u = df_train_X.groupby('product_id').apply(lambda x: len(x['user_id'].unique()))
# testデータにおける商品ごとのユーザ数
p_u_test = df_test_X.groupby('product_id').apply(lambda x: len(x['user_id'].unique()))
p_u.head(5)

product_id
00000001_b    2
00000002_b    1
00000004_b    1
00000005_b    2
00000007_b    1
dtype: int64

In [280]:
# 各商品に対してカートに入れられた回数
p_ca = df_train_X.groupby('product_id').apply(lambda x: (x['event_type']==0).sum())
# testデータにおける商品ごとのカートに入れられた回数
p_ca_test = df_test_X.groupby('product_id').apply(lambda x: (x['event_type']==0).sum())
p_ca.head(5)

product_id
00000001_b    0
00000002_b    0
00000004_b    0
00000005_b    0
00000007_b    0
dtype: int64

In [281]:
# 各商品に対して商品閲覧された回数
p_pv = df_train_X.groupby('product_id').apply(lambda x: (x['event_type']==1).sum())
# testデータにおける商品ごとの商品閲覧回数
p_pv_test = df_test_X.groupby('product_id').apply(lambda x: (x['event_type']==1).sum())
p_pv.head(5)

product_id
00000001_b    5
00000002_b    1
00000004_b    1
00000005_b    5
00000007_b    2
dtype: int64

In [282]:
# 各商品に対してクリックされた回数
p_cl = df_train_X.groupby('product_id').apply(lambda x: (x['event_type']==2).sum())
# testデータにおける商品ごとのクリック回数
p_cl_test = df_test_X.groupby('product_id').apply(lambda x: (x['event_type']==2).sum())
p_cl.head(5)

product_id
00000001_b    0
00000002_b    0
00000004_b    0
00000005_b    0
00000007_b    0
dtype: int64

In [283]:
# 各商品に対して購入された回数
p_cv = df_train_X.groupby('product_id').apply(lambda x: (x['event_type']==3).sum())
# testデータにおける商品ごとの購入回数
p_cv_test = df_test_X.groupby('product_id').apply(lambda x: (x['event_type']==3).sum())
p_cv.head(5)

product_id
00000001_b    0
00000002_b    0
00000004_b    0
00000005_b    0
00000007_b    0
dtype: int64

In [284]:
# 商品に関する特徴量を連結
p = pd.concat([p_u, p_ca, p_pv, p_cl, p_cv], axis=1)
p.columns = ['p_u', 'p_ca', 'p_pv', 'p_cl', 'p_cv']
# testデータにおける商品ごとのユーザ数，カートに入れられた回数，商品閲覧回数，クリック回数，購入回数
p_test = pd.concat([p_u_test, p_ca_test, p_pv_test, p_cl_test, p_cv_test], axis=1)
p_test.columns = ['p_u', 'p_ca', 'p_pv', 'p_cl', 'p_cv']
p.head(5)

,p_u,p_ca,p_pv,p_cl,p_cv
product_id,,,,,
00000001_b,2,0,5,0,0
00000002_b,1,0,1,0,0
00000004_b,1,0,1,0,0
00000005_b,2,0,5,0,0
00000007_b,1,0,2,0,0


In [285]:
p_test.head(5)

,p_u,p_ca,p_pv,p_cl,p_cv
product_id,,,,,
00000001_b,2,1,14,0,1
00000003_b,1,0,1,0,0
00000004_b,8,0,18,0,0
00000005_b,1,0,1,0,0
00000008_b,1,0,1,0,0


ユーザ×商品に関する特徴量を作成
> ユーザ×商品に対して閲覧された回数<br>
> ユーザ×商品に対してカートに入れられた回数<br>
> ユーザ×商品に対してクリックされた回数<br>
> ユーザ×商品に対して購入された回数<br>

In [286]:
# ユーザ×商品に対してカートに入れられた回数
u_p_ca = df_train_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==0).sum())
# testデータにおけるユーザ×商品ごとのカートに入れられた回数
u_p_ca_test = df_test_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==0).sum())
u_p_ca.head(5)

user_id    product_id
0000010_B  00007375_b    0
           00011168_b    0
           00011599_b    0
           00016319_b    0
0000018_B  00002919_b    0
dtype: int64

In [287]:
# ユーザ×商品に対して閲覧された回数
u_p_pv = df_train_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==1).sum())
# testデータにおけるユーザ×商品ごとの閲覧回数
u_p_pv_test = df_test_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==1).sum())
u_p_pv.head(5)

user_id    product_id
0000010_B  00007375_b    1
           00011168_b    1
           00011599_b    2
           00016319_b    2
0000018_B  00002919_b    1
dtype: int64

In [288]:
# ユーザ×商品に対してクリックされた回数
u_p_cl = df_train_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==2).sum())
# testデータにおけるユーザ×商品ごとのクリック回数
u_p_cl_test = df_test_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==2).sum())
u_p_cl.head(5)

user_id    product_id
0000010_B  00007375_b    0
           00011168_b    0
           00011599_b    0
           00016319_b    0
0000018_B  00002919_b    0
dtype: int64

In [289]:
# ユーザ×商品に対して購入された回数
u_p_cv = df_train_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==3).sum())
# testデータにおけるユーザ×商品ごとの購入回数
u_p_cv_test = df_test_X.groupby(['user_id', 'product_id']).apply(lambda x: (x['event_type']==3).sum())
u_p_cv.head(5)

user_id    product_id
0000010_B  00007375_b    0
           00011168_b    0
           00011599_b    0
           00016319_b    0
0000018_B  00002919_b    0
dtype: int64

In [290]:
# ユーザ×商品に関する特徴量を連結
u_p = pd.concat([u_p_ca, u_p_pv, u_p_cl, u_p_cv], axis=1)
u_p.columns = ['u_p_ca', 'u_p_pv', 'u_p_cl', 'u_p_cv']
# testデータにおけるユーザ×商品ごとのカートに入れられた回数，閲覧回数，クリック回数，購入回数
u_p_test = pd.concat([u_p_ca_test, u_p_pv_test, u_p_cl_test, u_p_cv_test], axis=1)
u_p_test.columns = ['u_p_ca', 'u_p_pv', 'u_p_cl', 'u_p_cv']
u_p.head(5)

u_p_ca  u_p_pv  u_p_cl  u_p_cv
user_id   product_id                                
0000010_B 00007375_b       0       1       0       0
          00011168_b       0       1       0       0
          00011599_b       0       2       0       0
          00016319_b       0       2       0       0
0000018_B 00002919_b       0       1       0       0

In [291]:
u_p_test.head(5)

u_p_ca  u_p_pv  u_p_cl  u_p_cv
user_id   product_id                                
0000010_B 00003227_b       0       1       0       0
          00003683_b       0       2       0       0
          00008693_b       0       3       0       0
          00009153_b       0       2       0       0
          00009248_b       0       3       0       0

今まで作成したユーザ，商品，ユーザ×商品データを１つに結合する<br>
マージが必要．連結との違いは共通とするカラムを指定し，それを軸として一つのデータにすることである．

In [292]:
# 変数uのuser_idインデックスをカラムに変更（インデックスリセット）
u.reset_index(inplace=True)
u.head(5)

,user_id,u_p,u_d,u_pv
0,0000010_B,4,1,6
1,0000018_B,8,3,10
2,0000033_B,19,4,87
3,0000036_B,1,1,1
4,0000040_B,13,4,43


In [293]:
# 変数u_testのuser_idインデックスをカラムに変更（インデックスリセット）
u_test.reset_index(inplace=True)
u_test.head(5)

,user_id,u_p,u_d,u_pv
0,0000010_B,9,1,23
1,0000015_B,1,1,1
2,0000018_B,4,4,7
3,0000041_B,2,1,1
4,0000045_B,11,2,23


In [295]:
# 変数pのproduct_idインデックスをカラムに変更（インデックスリセット）
p.reset_index(inplace=True)
p.head(5)

,product_id,p_u,p_ca,p_pv,p_cl,p_cv
0,00000001_b,2,0,5,0,0
1,00000002_b,1,0,1,0,0
2,00000004_b,1,0,1,0,0
3,00000005_b,2,0,5,0,0
4,00000007_b,1,0,2,0,0


In [296]:
# 変数p_testのproduct_idインデックスをカラムに変更（インデックスリセット）
p_test.reset_index(inplace=True)
p_test.head(5)

,product_id,p_u,p_ca,p_pv,p_cl,p_cv
0,00000001_b,2,1,14,0,1
1,00000003_b,1,0,1,0,0
2,00000004_b,8,0,18,0,0
3,00000005_b,1,0,1,0,0
4,00000008_b,1,0,1,0,0


In [297]:
# 変数u_pのuser_id, product_idインデックスをカラムに変更（インデックスリセット）
u_p.reset_index(inplace=True)
u_p.head(5)

,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv
0,0000010_B,00007375_b,0,1,0,0
1,0000010_B,00011168_b,0,1,0,0
2,0000010_B,00011599_b,0,2,0,0
3,0000010_B,00016319_b,0,2,0,0
4,0000018_B,00002919_b,0,1,0,0


In [298]:
# 変数u_p_testのuser_id, product_idインデックスをカラムに変更（インデックスリセット）
u_p_test.reset_index(inplace=True)
u_p_test.head(5)

,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv
0,0000010_B,00003227_b,0,1,0,0
1,0000010_B,00003683_b,0,2,0,0
2,0000010_B,00008693_b,0,3,0,0
3,0000010_B,00009153_b,0,2,0,0
4,0000010_B,00009248_b,0,3,0,0


In [299]:
# データu_pとデータuを結合(inner join)
merged = pd.merge(u_p, u, on='user_id', how='inner')
print(merged.shape)
merged.head(5)

(13552, 9)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv
0,0000010_B,00007375_b,0,1,0,0,4,1,6
1,0000010_B,00011168_b,0,1,0,0,4,1,6
2,0000010_B,00011599_b,0,2,0,0,4,1,6
3,0000010_B,00016319_b,0,2,0,0,4,1,6
4,0000018_B,00002919_b,0,1,0,0,8,3,10


In [300]:
# データu_p_testとデータu_testを結合(inner join)
merged_test = pd.merge(u_p_test, u_test, on='user_id', how='inner')
print(merged_test.shape)
merged_test.head(5)

(13358, 9)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv
0,0000010_B,00003227_b,0,1,0,0,9,1,23
1,0000010_B,00003683_b,0,2,0,0,9,1,23
2,0000010_B,00008693_b,0,3,0,0,9,1,23
3,0000010_B,00009153_b,0,2,0,0,9,1,23
4,0000010_B,00009248_b,0,3,0,0,9,1,23


In [301]:
# mergedとデータpを結合(inner join)
merged = pd.merge(merged, p, on='product_id', how='inner')
print(merged.shape)
merged.head(5)
# これにより訓練データの説明変数が完成

(13552, 14)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv
0,0000010_B,00007375_b,0,1,0,0,4,1,6,4,0,5,0,0
1,0003325_B,00007375_b,0,1,0,0,7,4,11,4,0,5,0,0
2,0011285_B,00007375_b,0,1,0,0,30,5,103,4,0,5,0,0
3,0022312_B,00007375_b,0,2,0,0,16,2,29,4,0,5,0,0
4,0000010_B,00011168_b,0,1,0,0,4,1,6,2,0,4,0,0


In [302]:
# merged_testとデータp_testを結合(inner join)
merged_test = pd.merge(merged_test, p_test, on='product_id', how='inner')
print(merged_test.shape)
merged_test.head(5)

(13358, 14)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv
0,0000010_B,00003227_b,0,1,0,0,9,1,23,1,0,1,0,0
1,0000010_B,00003683_b,0,2,0,0,9,1,23,4,3,9,0,1
2,0004021_B,00003683_b,3,1,0,1,10,5,29,4,3,9,0,1
3,0010684_B,00003683_b,0,1,0,0,17,1,19,4,3,9,0,1
4,0021459_B,00003683_b,0,5,0,0,16,3,35,4,3,9,0,1


### 目的変数データを作成

In [303]:
# ユーザと商品の各ペアに対して関連度を付与することで目的変数データを作成
# 関連度はその商品に対するユーザの興味の度合いを表す
# 関連度はevent_typeの最も大きな値に1を足したものとし，関連度が0の場合は関連度なし.
# ユーザと商品の各ペアに対して関連度を付与し，変数relに代入
rel = df_train_y.groupby(['user_id', 'product_id']).apply(lambda x: max(x['event_type'])+1)
rel.head(5)

user_id    product_id
0000015_B  00000744_b    4
           00000841_b    4
           00001763_b    4
           00003399_b    2
           00004391_b    4
dtype: int64

In [304]:
# テストデータのユーザと商品の各ペアに対して関連度を付与
rel_test = df_test_y.groupby(['user_id', 'product_id']).apply(lambda x: max(x['event_type'])+1)
rel_test.head(5)

user_id    product_id
0000015_B  00004792_b    2
           00006757_b    2
           00016863_b    2
0000018_B  00005909_b    2
           00006025_b    2
dtype: int64

In [305]:
type(rel)

pandas.core.series.Series

In [306]:
# relのインデックスをカラムに変更（インデックスリセット）
# rel.reset_index(inplace=True)ではSeriesを変換するということでエラーが出る
rel = rel.reset_index()
rel.head(5)

,user_id,product_id,0
0,0000015_B,00000744_b,4
1,0000015_B,00000841_b,4
2,0000015_B,00001763_b,4
3,0000015_B,00003399_b,2
4,0000015_B,00004391_b,4


In [307]:
# rel_testのインデックスをカラムに変更（インデックスリセット）
rel_test = rel_test.reset_index()
rel_test.head(5)

,user_id,product_id,0
0,0000015_B,00004792_b,2
1,0000015_B,00006757_b,2
2,0000015_B,00016863_b,2
3,0000018_B,00005909_b,2
4,0000018_B,00006025_b,2


In [308]:
rel.rename(columns={0: 'y'}, inplace=True)
rel.head(5)

,user_id,product_id,y
0,0000015_B,00000744_b,4
1,0000015_B,00000841_b,4
2,0000015_B,00001763_b,4
3,0000015_B,00003399_b,2
4,0000015_B,00004391_b,4


In [309]:
# rel_testのカラム名を変更
rel_test.rename(columns={0: 'y'}, inplace=True)
rel_test.head(5)

,user_id,product_id,y
0,0000015_B,00004792_b,2
1,0000015_B,00006757_b,2
2,0000015_B,00016863_b,2
3,0000018_B,00005909_b,2
4,0000018_B,00006025_b,2


訓練データ，テストデータの両方に存在するユーザを抽出．<br>
訓練できないため. やり方としてはset関数で集合データとして抽出し，共通部分を抽出する

In [310]:
# 説明変数のユーザ
user_X_train = set(merged['user_id'])
# 目的変数のユーザ
user_y_train = set(rel['user_id'])
# 共通のユーザを抽出
user_Xy_train = user_X_train.intersection(user_y_train)
# ユーザの数
print(len(user_X_train), len(user_y_train), len(user_Xy_train))
# 975人が被っている

1446 1684 975


In [311]:
# テストデータに関するユーザのデータも同様に作成
# 説明変数のユーザ
user_X_test = set(merged_test['user_id'])
# 目的変数のユーザ
user_y_test = set(rel_test['user_id'])
# 共通のユーザを抽出
user_Xy_test = user_X_test.intersection(user_y_test)

In [312]:
# 特徴量データに目的変数データを紐付ける．
# 特徴量データから共通のユーザに関するデータのみ抽出(今回みたいにユーザーが複数の場合はisinメソッドを使う)
merged = merged[merged['user_id'].isin(user_Xy_train)]
print(merged.shape)
merged.head(5)

(7967, 14)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv
1,0003325_B,00007375_b,0,1,0,0,7,4,11,4,0,5,0,0
2,0011285_B,00007375_b,0,1,0,0,30,5,103,4,0,5,0,0
5,0014454_B,00011168_b,0,3,0,0,43,7,103,2,0,4,0,0
10,0004576_B,00016319_b,0,2,0,0,19,4,53,3,0,10,0,0
11,0014454_B,00016319_b,0,6,0,0,43,7,103,3,0,10,0,0


In [313]:
# 特徴量データから共通のユーザに関するデータのみ抽出
merged_test = merged_test[merged_test['user_id'].isin(user_Xy_test)]
print(merged_test.shape)
merged_test.head(5)

(7035, 14)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv
2,0004021_B,00003683_b,3,1,0,1,10,5,29,4,3,9,0,1
4,0021459_B,00003683_b,0,5,0,0,16,3,35,4,3,9,0,1
6,0021459_B,00008693_b,0,7,0,0,16,3,35,2,0,10,0,0
13,0021274_B,00010114_b,0,3,0,0,5,2,17,4,0,20,0,0
18,0000015_B,00000744_b,1,1,0,1,1,1,1,2,1,2,0,1


In [314]:
# 目的変数データから特徴量データに存在するユーザと商品のペアを抽出
# この順番でないと，学習データには存在するが，テストデータには存在しないユーザと商品のペアが存在してしまう
# 目的変数の抽出. inner joinを使用することで，特徴量データに存在するユーザと商品のペアのみ抽出
rel = pd.merge(merged[['user_id', 'product_id']],rel, on=['user_id', 'product_id'], how='inner')
rel.head(5)

,user_id,product_id,y
0,0004576_B,00016319_b,2
1,0000018_B,00006025_b,3
2,0007555_B,00007187_b,2
3,0009087_B,00007187_b,2
4,0019599_B,00007187_b,2


In [315]:
# 目的変数データから特徴量データに存在するユーザと商品のペアを抽出
rel_test = pd.merge(merged_test[['user_id', 'product_id']],rel_test, on=['user_id', 'product_id'], how='inner')
rel_test.head(5)

,user_id,product_id,y
0,0004021_B,00003683_b,2
1,0021459_B,00008693_b,2
2,0000018_B,00005909_b,2
3,0000041_B,00002768_b,2
4,0000041_B,00013590_b,2


In [316]:
# その次にマージ．今回はouter joinを用いる．
# outer joinは指定したカラムにおいて２つのデータのうち少なくとも一方に存在する行を結合する方法．空いた場所は欠損値となる
train_all = pd.merge(merged, rel, on=['user_id', 'product_id'], how='outer')
print(train_all.shape)
train_all.head(5)

(7967, 15)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv,y
0,0003325_B,00007375_b,0,1,0,0,7,4,11,4,0,5,0,0,NaN
1,0011285_B,00007375_b,0,1,0,0,30,5,103,4,0,5,0,0,NaN
2,0014454_B,00011168_b,0,3,0,0,43,7,103,2,0,4,0,0,NaN
3,0004576_B,00016319_b,0,2,0,0,19,4,53,3,0,10,0,0,2.0
4,0014454_B,00016319_b,0,6,0,0,43,7,103,3,0,10,0,0,NaN


In [317]:
# その次にマージ．今回はouter joinを用いる．
# outer joinは指定したカラムにおいて２つのデータのうち少なくとも一方に存在する行を結合する方法．空いた場所は欠損値となる
test_all = pd.merge(merged_test, rel_test, on=['user_id', 'product_id'], how='outer')
print(test_all.shape)
test_all.head(5)

(7035, 15)


,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv,y
0,0004021_B,00003683_b,3,1,0,1,10,5,29,4,3,9,0,1,2.0
1,0021459_B,00003683_b,0,5,0,0,16,3,35,4,3,9,0,1,NaN
2,0021459_B,00008693_b,0,7,0,0,16,3,35,2,0,10,0,0,2.0
3,0021274_B,00010114_b,0,3,0,0,5,2,17,4,0,20,0,0,NaN
4,0000015_B,00000744_b,1,1,0,1,1,1,1,2,1,2,0,1,NaN


In [318]:
# 欠損値を0で埋める
train_all.fillna(0, inplace=True)
train_all.head(5)

,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv,y
0,0003325_B,00007375_b,0,1,0,0,7,4,11,4,0,5,0,0,0.0
1,0011285_B,00007375_b,0,1,0,0,30,5,103,4,0,5,0,0,0.0
2,0014454_B,00011168_b,0,3,0,0,43,7,103,2,0,4,0,0,0.0
3,0004576_B,00016319_b,0,2,0,0,19,4,53,3,0,10,0,0,2.0
4,0014454_B,00016319_b,0,6,0,0,43,7,103,3,0,10,0,0,0.0


In [319]:
# 欠損値を0で埋める
test_all.fillna(0, inplace=True)
test_all.head(5)

,user_id,product_id,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv,y
0,0004021_B,00003683_b,3,1,0,1,10,5,29,4,3,9,0,1,2.0
1,0021459_B,00003683_b,0,5,0,0,16,3,35,4,3,9,0,1,0.0
2,0021459_B,00008693_b,0,7,0,0,16,3,35,2,0,10,0,0,2.0
3,0021274_B,00010114_b,0,3,0,0,5,2,17,4,0,20,0,0,0.0
4,0000015_B,00000744_b,1,1,0,1,1,1,1,2,1,2,0,1,0.0


In [320]:
# 関連度を整数に変換
train_all['y'] = train_all['y'].astype(int)

In [321]:
# 関連度を整数に変換
test_all['y'] = test_all['y'].astype(int)

機械学習モデルへ入力するためのデータ作成

In [322]:
print(train_all.shape)
print(test_all.shape)

(7967, 15)
(7035, 15)


In [323]:
# 各ユーザごとに存在する商品の数を列挙したクエリリストデータ作成
# 作成意図：説明変数データと目的変数データがクエリごと(ユーザごと)に区切られて学習を行わせるため
# user_id, product_idをインデックスに設定．説明変数に含めないため
# クエリリストデータの作成
query_list = train_all['user_id'].groupby(train_all['user_id']).count()
query_list.head(5)

user_id
0000018_B     8
0000036_B     1
0000040_B    13
0000041_B     4
0000049_B     3
Name: user_id, dtype: int64

In [324]:
# クエリリストデータの作成
query_list_test = test_all['user_id'].groupby(test_all['user_id']).count()
query_list_test.head(5)

user_id
0000015_B    1
0000018_B    4
0000041_B    2
0000055_B    2
0000072_B    1
Name: user_id, dtype: int64

In [325]:
# クエリリストをインデックスでソート
query_list.sort_index(inplace=True)
query_list.head(5)

user_id
0000018_B     8
0000036_B     1
0000040_B    13
0000041_B     4
0000049_B     3
Name: user_id, dtype: int64

In [326]:
# クエリリストをインデックスでソート
query_list_test.sort_index(inplace=True)
query_list_test.head(5)

user_id
0000015_B    1
0000018_B    4
0000041_B    2
0000055_B    2
0000072_B    1
Name: user_id, dtype: int64

In [327]:
# user_id, product_idをインデックス化
train_all.set_index(['user_id', 'product_id'], inplace=True)
train_all.head(5)

,,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv,y
user_id,product_id,,,,,,,,,,,,,
0003325_B,00007375_b,0,1,0,0,7,4,11,4,0,5,0,0,0
0011285_B,00007375_b,0,1,0,0,30,5,103,4,0,5,0,0,0
0014454_B,00011168_b,0,3,0,0,43,7,103,2,0,4,0,0,0
0004576_B,00016319_b,0,2,0,0,19,4,53,3,0,10,0,0,2
0014454_B,00016319_b,0,6,0,0,43,7,103,3,0,10,0,0,0


In [328]:
# user_id, product_idをインデックス化
test_all.set_index(['user_id', 'product_id'], inplace=True)
test_all.head(5)

u_p_ca  u_p_pv  u_p_cl  u_p_cv  u_p  u_d  u_pv  p_u  \
user_id   product_id                                                        
0004021_B 00003683_b       3       1       0       1   10    5    29    4   
0021459_B 00003683_b       0       5       0       0   16    3    35    4   
          00008693_b       0       7       0       0   16    3    35    2   
0021274_B 00010114_b       0       3       0       0    5    2    17    4   
0000015_B 00000744_b       1       1       0       1    1    1     1    2   

                      p_ca  p_pv  p_cl  p_cv  y  
user_id   product_id                             
0004021_B 00003683_b     3     9     0     1  2  
0021459_B 00003683_b     3     9     0     1  0  
          00008693_b     0    10     0     0  2  
0021274_B 00010114_b     0    20     0     0  0  
0000015_B 00000744_b     1     2     0     1  0

In [329]:
# 説明変数をインデックスでソート
train_all.sort_index(inplace=True)
train_all.head(5)

u_p_ca  u_p_pv  u_p_cl  u_p_cv  u_p  u_d  u_pv  p_u  \
user_id   product_id                                                        
0000018_B 00002919_b       0       1       0       0    8    3    10    1   
          00004816_b       0       1       0       0    8    3    10    2   
          00006025_b       0       1       0       0    8    3    10    1   
          00007187_b       0       2       0       0    8    3    10    8   
          00008466_b       0       1       0       0    8    3    10    1   

                      p_ca  p_pv  p_cl  p_cv  y  
user_id   product_id                             
0000018_B 00002919_b     0     1     0     0  0  
          00004816_b     0     2     0     0  0  
          00006025_b     0     1     0     0  3  
          00007187_b     0    16     0     0  0  
          00008466_b     0     1     0     0  0

In [330]:
# 説明変数をインデックスでソート
test_all.sort_index(inplace=True)
test_all.head(5)

u_p_ca  u_p_pv  u_p_cl  u_p_cv  u_p  u_d  u_pv  p_u  \
user_id   product_id                                                        
0000015_B 00000744_b       1       1       0       1    1    1     1    2   
0000018_B 00001352_b       0       1       0       0    4    4     7    1   
          00003021_b       0       2       1       0    4    4     7    1   
          00005909_b       0       2       0       0    4    4     7    2   
          00008500_b       0       2       1       0    4    4     7    1   

                      p_ca  p_pv  p_cl  p_cv  y  
user_id   product_id                             
0000015_B 00000744_b     1     2     0     1  0  
0000018_B 00001352_b     0     1     0     0  0  
          00003021_b     0     2     1     0  0  
          00005909_b     0     3     0     0  2  
          00008500_b     0     2     1     0  0

In [337]:
# yカラム以外の説明変数を抽出
X_train = train_all.drop('y', axis=1)
print(X_train.shape)
X_train.head(5)

(7967, 12)


u_p_ca  u_p_pv  u_p_cl  u_p_cv  u_p  u_d  u_pv  p_u  \
user_id   product_id                                                        
0000018_B 00002919_b       0       1       0       0    8    3    10    1   
          00004816_b       0       1       0       0    8    3    10    2   
          00006025_b       0       1       0       0    8    3    10    1   
          00007187_b       0       2       0       0    8    3    10    8   
          00008466_b       0       1       0       0    8    3    10    1   

                      p_ca  p_pv  p_cl  p_cv  
user_id   product_id                          
0000018_B 00002919_b     0     1     0     0  
          00004816_b     0     2     0     0  
          00006025_b     0     1     0     0  
          00007187_b     0    16     0     0  
          00008466_b     0     1     0     0

In [338]:
# yカラム以外の説明変数を抽出
X_test = test_all.drop('y', axis=1)
print(X_test.shape)
X_test.head(5)

(7035, 12)


u_p_ca  u_p_pv  u_p_cl  u_p_cv  u_p  u_d  u_pv  p_u  \
user_id   product_id                                                        
0000015_B 00000744_b       1       1       0       1    1    1     1    2   
0000018_B 00001352_b       0       1       0       0    4    4     7    1   
          00003021_b       0       2       1       0    4    4     7    1   
          00005909_b       0       2       0       0    4    4     7    2   
          00008500_b       0       2       1       0    4    4     7    1   

                      p_ca  p_pv  p_cl  p_cv  
user_id   product_id                          
0000015_B 00000744_b     1     2     0     1  
0000018_B 00001352_b     0     1     0     0  
          00003021_b     0     2     1     0  
          00005909_b     0     3     0     0  
          00008500_b     0     2     1     0

In [339]:
y_train = train_all['y']
print(y_train.shape)
y_train.head(5)

(7967,)


user_id    product_id
0000018_B  00002919_b    0
           00004816_b    0
           00006025_b    3
           00007187_b    0
           00008466_b    0
Name: y, dtype: int64

In [340]:
# 目的変数の抽出
y_test = test_all['y']
print(y_test.shape)
y_test.head(5)

(7035,)


user_id    product_id
0000015_B  00000744_b    0
0000018_B  00001352_b    0
           00003021_b    0
           00005909_b    2
           00008500_b    0
Name: y, dtype: int64

機械学習モデルを学習させる

目的変数がカテゴリ変数の場合：分類問題
目的変数が実数：回帰問題
本課題ではカテゴリ変数ではあるが実際に数値に順序は存在しているため分類問題と回帰問題の両方の特徴を持っているように思う．このような問題にはランク学習という学習モデルが適している．

In [335]:
import lightgbm as lgb
model = lgb.LGBMRanker()

In [235]:
print("Length of eval_set:", len([(X_test, y_test)]))
print("Length of eval_group:", len([list(query_list_test)]))


Length of eval_set: 1
Length of eval_group: 1


In [236]:
print("Total samples in X_test:", len(X_test))
print("Total samples in query_list_test:", sum(query_list_test))


Total samples in X_test: 7035
Total samples in query_list_test: 7035


In [237]:
print("First few elements of query_list_test:", query_list_test[:10])


First few elements of query_list_test: user_id
0000015_B    1
0000018_B    4
0000041_B    2
0000055_B    2
0000072_B    1
0000074_B    5
0000084_B    2
0000087_B    2
0000089_B    1
0000092_B    2
Name: user_id, dtype: int64


In [239]:
list(query_list_test)

978

In [242]:
sum(list(query_list))

7967

In [342]:
print(X_train.shape)
print(X_test.shape)

(7967, 12)
(7035, 12)


In [343]:
# モデルの学習
# eval_groupにはリストのリストを渡す必要がある
model.fit(X_train, y_train, group=list(query_list), eval_set=[(X_test, y_test)], eval_group=[list(query_list_test)])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002126 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 291
[LightGBM] [Info] Number of data points in the train set: 7967, number of used features: 12


LGBMRanker()

ユーザに対しての予測

In [350]:
X_test.loc['0000018_B']
# 商品が4つということがわかる．このため予測数は4つになる

,u_p_ca,u_p_pv,u_p_cl,u_p_cv,u_p,u_d,u_pv,p_u,p_ca,p_pv,p_cl,p_cv
product_id,,,,,,,,,,,,
00001352_b,0,1,0,0,4,4,7,1,0,1,0,0
00003021_b,0,2,1,0,4,4,7,1,0,2,1,0
00005909_b,0,2,0,0,4,4,7,2,0,3,0,0
00008500_b,0,2,1,0,4,4,7,1,0,2,1,0


In [351]:
# modelのユーザーID'0000018_Bに対する出力結果
pred = model.predict(X_test.loc['0000018_B'])
pred

array([-0.70514921, -0.47941292, -0.65304016, -0.47941292])

In [352]:
# predに対して商品IDを付与
pred_products = pd.Series(pred, index=X_test.loc['0000018_B'].index)
pred_products

product_id
00001352_b   -0.705149
00003021_b   -0.479413
00005909_b   -0.653040
00008500_b   -0.479413
dtype: float64

In [355]:
# 商品を関連度順にソート
products = pred_products.sort_values(ascending=False).index # 降順にしたいから
products # 大きい順のproduct_idが出力される

Index(['00003021_b', '00008500_b', '00005909_b', '00001352_b'], dtype='object', name='product_id')

In [357]:
# 上位5件を出力
output = list(products)[:5]
output

['00003021_b', '00008500_b', '00005909_b', '00001352_b']

精度評価(nDCG)<br>
推薦エンジンの精度評価によく用いられる評価指標